# 구문 분석: Part II - Context Free Grammar

All rights reserved, 2021, By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 참고 사이트 
    - nltk book 8.Analyzing sentence structure(https://www.nltk.org/book/ch08.html) 내용을 참고해서 자료를 만듦. 
    - nltk book의 8장 예제를 일부 사용.
- 참고 서적
    - Natural Language Processing with Python Cookbook, Krishna Bhavsar, Naresh Kumar, Pratap Dangeti, Packt Publishing, 2017.
    - Chapter 6, Chapter 7의 예제를 일부 사용.

여기서 다룰 내용은 아래 질문에 대한 답입니다.
- 다양한 문장 구조를 표현하려면 형식 문법을 어떻게 정의하면 될까? 
- 문장 분석 과정을 구문 트리에서 어떻게 보여줄 수 있을까?
- 파서(parser)가 문장을 어떻게 분석하고, 구문 트리를 자동으로 만들 수 있을까?

In [ ]:
import nltk, re
from nltk.parse.generate import generate

## 2. 문맥 자유 문법(Context-Free Grammar )

## 2.1   간단한 CFG 
- 문맥 자유 문법(context free grammar, cfg)에서는 생성규칙의 왼쪽(left-hand-side, lhs)에 놓인 기호는 nonterminal 기호 하나뿐입니다.
- CFG에서 첫 번째 생성규칙의 왼쪽(lhs)에 놓인 기호가 시작기호(start symbol) S 이다. S는 구문 트리의 루트(single root)가 됩니다.
- NLTK에서 CFG는 nltk.grammar 모듈에 정의되어 있습니다.

In [ ]:
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
""")

nltk.app.rdparser()를 사용해서 recursive descent parser(**rdparser**)의 파싱 과정을 단계별로 확인할 수 있습니다. 
- Edit : (이미 샘플 문법이 있음) -> edit grammar -> 문법 편집(또는 위 셀에 있는 내용을 복사해서 붙여넣기해도 됨)
- autostep : 파서 실행(step으로 한 단계씩 실행시킬 수도 있음)
- 아래 셀에서 주석을 없애고 직접 실행시켜 보세요.

In [ ]:
#nltk.app.rdparser()

split() 메소드를 사용하여 문장에서 토큰(token)을 찾습니다.

In [ ]:
sent = "the dog saw a man in the park".split() 
sent

위 문법은 샘플 문장에 대해 2개의 트리를 생성하기 때문에, 이 문장은 모호한 문장(2가지 의미로 해석)입니다. 
- 모호함이 발생한 이유는 전치사구(PP) in the park 때문입니다. 
    - 즉 PP(in the park)가 NP의 child가 될 수도 아니면 VP의 child가 될 수도 있기 때문입니다.
    - PP가 NP의 child가 되면, 주인공 개가 다른 곳에서 공원에 있는 남자를 본 것으로 해석됩니다.
    - PP가 VP의 child가 되면, 주인공 개가 같이 공원에 있으면서 남자를 본 것으로 해석됩니다.

In [ ]:
parser = nltk.RecursiveDescentParser(grammar1)
for tree in parser.parse(sent):
    print(tree)
    #tree.draw()

전치사구(PP)를 제거한 문장은 1개 트리만 생성하기 때문에 모호함이 사라졌습니다. 

In [ ]:
sent = "the dog saw a man".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print(tree)
    #tree.draw()    

## 2.2   파일에서 문법 읽어오기
- 문자열로 작성한 문법은 *메모장(notepad)* 을 사용하여 텍스트 파일로 저장할 수 있습니다. 
    - *grammar1* 에서 따옴표를 제외한 생성규칙을 복사한 다음 파일로 저장하세요.
        - jupyter notebook 파일과 같은 폴더에 저장하세요.
        - 파일 이름은 mygrammar.cfg 입니다.
        - 파일 확장자(suffix)는 반드시 '.cfg'로 저장해야 합니다.
- 파일에 저장한 문법은 nltk.data.load() 로 읽어올 수 있습니다. 
    - 'file:mygrammar.cfg'에 공백(space)이 있으면 안됩니다.

In [ ]:
grammar1 = nltk.data.load('file:mygrammar.cfg')

In [ ]:
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print(tree)
    #tree.draw()    

만약 아래처럼 아무 것도 출력되지 않으면, 문장(sent)에 syntax error가 있을 가능성이 높습니다. 

In [ ]:
sent = "Mary Bob saw ".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print(tree)

이 경우 parser가 구문 분석 과정을 추적(trace)할 수 있도록 옵션(trace=2)을 설정해 보겠습니다.

In [ ]:
sent = "Mary Bob saw ".split()
rd_parser = nltk.RecursiveDescentParser(grammar1, trace=2)
for tree in rd_parser.parse(sent):
    print(tree)

- syntax error가 발생한 경우 생성 규칙에 문제가 있는지 확인하는 것이 대표적 해결 방법입니다.
- 아래 예는 첫 번째 생성 규칙부터 10번째 생성규칙까지 10개만 출력합니다.
    - A -> B | C | D 는 생성규칙 1개가 아니라 모두 3개 생성규칙이 있습니다.

In [ ]:
for p in grammar1.productions()[:10]:  
    print(p)

NLTK에서 문법을 작성할 때, 몇 가지 주의할 점.
- 생성 규칙 오른쪽(rhs)에 terminal 기호와 nonterminal 기호를 함께 사용할 수 없습니다.
    - **PP -> 'of' NP** 와 같이 쓸 수 없습니다.
- 또, 다중 토큰(multi-word lexical items)을 사용해서도 안된다. 
    - **NP -> 'New York'** 대신 **NP -> 'New_York'** 와 같이 단일 토큰 표현을 써야 합니다.

## 2.3   순환 생성 규칙 

문법의 생성 규칙은 순환(recursive) 구조를 갖고 있습니다. 
- **Nom -> Adj Nom** 처럼 생성 규칙 왼쪽(lhs)의 nonterminal이 오른쪽(rhs)에 다시 나타나는 경우를 말합니다.
    - 이런 순환을 직접 순환(direct recursion)이라 합니다.
- **S -> NP VP** 와 **VP -> V S** 처럼 문법 형태에서 간접 순환(indirect recursion)도 자주 등장합니다. 

- 아래 grammar2에서는 어디 어디서 순환이 정의되어 있을까요?
    - 직접 순환과 간접 순환을 모두 찾아보기 바랍니다.

In [ ]:
grammar2 = nltk.CFG.fromstring("""
  S  -> NP VP
  NP -> Det Nom | PropN
  Nom -> Adj Nom | N
  VP -> V Adj | V NP | V S | V NP PP
  PP -> P NP
  PropN -> 'Buster' | 'Chatterer' | 'Joe'
  Det -> 'the' | 'a'
  N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
  Adj  -> 'angry' | 'frightened' |  'little' | 'tall'
  V ->  'chased'  | 'saw' | 'said' | 'thought' | 'was' | 'put'
  P -> 'on'
  """)

아래 syntax tree의 깊이(depth)는 8입니다.

In [ ]:
rd_parser = nltk.RecursiveDescentParser(grammar2)
sent = 'the angry bear chased the frightened little squirrel'.split()
for tree in rd_parser.parse(sent):
    print(tree)
    tree.draw()

아래 syntax tree의 깊이(depth)는 9입니다.

In [ ]:
rd_parser = nltk.RecursiveDescentParser(grammar2)
sent = 'Chatterer said Buster thought the tree was tall'.split()
for tree in rd_parser.parse(sent):
    print(tree)
    tree.draw()

### 꼭! 꼭! 기억하기 바랍니다.
- RecursiveDescentParser는 X -> X Y와 같은 좌순환(left-recursive) 생성 규칙을 다룰 수 없습니다. 
- grammar2의 순환은 모두 우순환(right-recursive)하다는 점을 확인하세요.